<a href="https://colab.research.google.com/github/aish2509/Mentalhealthcompanion/blob/main/finalcode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# --- Install dependencies ---
!pip install datasets tensorflow numpy pandas nltk --quiet

# --- Import libraries ---
import pandas as pd
import numpy as np
import tensorflow as tf
from datasets import load_dataset
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import layers, regularizers
from nltk.translate.bleu_score import sentence_bleu
import re
import random
from nltk import word_tokenize
import nltk

nltk.download('punkt_tab')

# --- Load and clean datasets ---
amod = load_dataset("Amod/mental_health_counseling_conversations")
alex = load_dataset("alexandreteles/mental-health-conversational-data")

amod_df = amod['train'].to_pandas()[['Context', 'Response']]
alex_df = alex['train'].to_pandas()[['Context', 'Response']]

# Clean text
def clean_text(text):
    text = text.lower().strip()
    text = re.sub(r'[^\w\s.,!?]', '', text)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r"(\b[iI])'([mM]\b|[lL]{2}\b|[vV][eE]\b)", r"\1'\2", text)
    return text

amod_df['Context'] = amod_df['Context'].apply(clean_text)
amod_df['Response'] = amod_df['Response'].apply(clean_text)
alex_df['Context'] = alex_df['Context'].apply(clean_text)
alex_df['Response'] = alex_df['Response'].apply(clean_text)

# Filter short or empty responses
amod_df = amod_df[amod_df['Context'].str.len() > 10]
amod_df = amod_df[amod_df['Response'].str.len() > 20]
alex_df = alex_df[alex_df['Context'].str.len() > 10]
alex_df = alex_df[alex_df['Response'].str.len() > 20]

# Combine datasets
combined_df = pd.concat([amod_df, alex_df], ignore_index=True)
print(f"Total training samples: {len(combined_df)}")

# --- Data augmentation ---
def paraphrase_sentence(sentence):
    words = word_tokenize(sentence)
    if len(words) < 5:
        return sentence
    synonyms = {
        'sad': ['unhappy', 'down', 'blue'],
        'happy': ['joyful', 'content', 'pleased'],
        'anxious': ['nervous', 'worried', 'tense'],
        'help': ['support', 'aid', 'assistance'],
        'feel': ['sense', 'experience', 'perceive']
    }
    new_words = words.copy()
    for i, word in enumerate(words):
        if word in synonyms and random.random() < 0.3:
            new_words[i] = random.choice(synonyms[word])
    return ' '.join(new_words)

augmented_data = []
for _, row in combined_df.iterrows():
    augmented_data.append({'Context': row['Context'], 'Response': row['Response']})
    if random.random() < 0.5:
        augmented_data.append({
            'Context': paraphrase_sentence(row['Context']),
            'Response': paraphrase_sentence(row['Response'])
        })
combined_df = pd.DataFrame(augmented_data)
print(f"Total samples after augmentation: {len(combined_df)}")

# --- Prepare text data ---
max_vocab = 15000  # Increased vocabulary size
max_len_src = 60   # Adjusted based on dataset analysis
max_len_tgt = 60
embedding_dim = 256
lstm_units = 512
dropout_rate = 0.3
batch_size = 64
epochs = 10
initial_learning_rate = 1e-3

# Add start/end tokens
combined_df['response_input'] = '<start> ' + combined_df['Response']
combined_df['response_output'] = combined_df['Response'] + ' <end>'

# Tokenization
tokenizer = Tokenizer(num_words=max_vocab, oov_token='<OOV>',
                      filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
all_texts = combined_df['Context'].tolist() + combined_df['response_input'].tolist() + combined_df['response_output'].tolist()
tokenizer.fit_on_texts(all_texts)
word_index = tokenizer.word_index
vocab_size = min(max_vocab, len(word_index) + 1)
print(f"Vocabulary size: {vocab_size}")

if '<start>' not in word_index:
    word_index['<start>'] = len(word_index) + 1
if '<end>' not in word_index:
    word_index['<end>'] = len(word_index) + 1

# Convert texts to sequences
encoder_input = tokenizer.texts_to_sequences(combined_df['Context'].tolist())
decoder_input = tokenizer.texts_to_sequences(combined_df['response_input'].tolist())
decoder_output = tokenizer.texts_to_sequences(combined_df['response_output'].tolist())

encoder_input = pad_sequences(encoder_input, maxlen=max_len_src, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen=max_len_tgt, padding='post')
decoder_output = pad_sequences(decoder_output, maxlen=max_len_tgt, padding='post')
decoder_output_exp = np.expand_dims(decoder_output, -1)

print(f"Encoder input shape: {encoder_input.shape}")
print(f"Decoder input shape: {decoder_input.shape}")
print(f"Decoder output shape: {decoder_output_exp.shape}")

# --- Build model with improved architecture ---
# Encoder
encoder_inputs = tf.keras.Input(shape=(max_len_src,))
enc_emb_layer = layers.Embedding(vocab_size, embedding_dim)
enc_emb = enc_emb_layer(encoder_inputs)
enc_emb = layers.BatchNormalization()(enc_emb)
encoder_lstm = layers.Bidirectional(
    layers.LSTM(lstm_units, return_sequences=True, return_state=True,
                dropout=dropout_rate, recurrent_dropout=dropout_rate,
                kernel_regularizer=regularizers.l2(1e-5))
)
encoder_outputs, forward_h, forward_c, backward_h, backward_c = encoder_lstm(enc_emb)
state_h = layers.Concatenate()([forward_h, backward_h])
state_c = layers.Concatenate()([forward_c, backward_c])
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = tf.keras.Input(shape=(max_len_tgt,))
dec_emb_layer = layers.Embedding(vocab_size, embedding_dim)
dec_emb = dec_emb_layer(decoder_inputs)
dec_emb = layers.BatchNormalization()(dec_emb)
decoder_lstm = layers.LSTM(
    lstm_units * 2, return_sequences=True, return_state=True,
    dropout=dropout_rate, recurrent_dropout=dropout_rate,
    kernel_regularizer=regularizers.l2(1e-5)
)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)

# Multi-Head Attention
attention_layer = layers.MultiHeadAttention(num_heads=8, key_dim=64)
attention_output = attention_layer(decoder_outputs, encoder_outputs)
decoder_concat_input = layers.Concatenate(axis=-1)([decoder_outputs, attention_output])
decoder_concat_input = layers.LayerNormalization()(decoder_concat_input)

# Dense layer
decoder_dense = layers.TimeDistributed(
    layers.Dense(vocab_size, activation='softmax',
                 kernel_regularizer=regularizers.l2(1e-5))
)
decoder_outputs_final = decoder_dense(decoder_concat_input)

# Compile model
model = tf.keras.Model([encoder_inputs, decoder_inputs], decoder_outputs_final)
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=initial_learning_rate),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Learning rate schedule
lr_schedule = tf.keras.optimizers.schedules.CosineDecay(
    initial_learning_rate, decay_steps=epochs * len(combined_df) // batch_size, alpha=1e-6
)
# The optimizer's learning rate is not settable when a schedule is used.
# Remove this line to allow ReduceLROnPlateau to adjust the learning rate.
# model.optimizer.learning_rate = lr_schedule


# Callbacks
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6, verbose=1
)
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=7, restore_best_weights=True, verbose=1
)

# BLEU score callback
class BLEUCallback(tf.keras.callbacks.Callback):
    def __init__(self, validation_data, index_word):
        super().__init__()
        self.validation_data = validation_data
        self.index_word = index_word

    def on_epoch_end(self, epoch, logs=None):
        encoder_input_val, decoder_input_val, decoder_output_val = self.validation_data
        bleu_scores = []
        for i in range(min(10, len(encoder_input_val))):
            pred_seq = self.model.predict([encoder_input_val[i:i+1], decoder_input_val[i:i+1]], verbose=0)
            pred_ids = np.argmax(pred_seq, axis=-1)[0]
            ref_ids = decoder_output_val[i].flatten()
            pred_text = [self.index_word.get(id, '') for id in pred_ids if id > 0]
            ref_text = [self.index_word.get(id, '') for id in ref_ids if id > 0]
            if len(pred_text) > 0 and len(ref_text) > 0:
                bleu_scores.append(sentence_bleu([ref_text], pred_text, weights=(0.5, 0.5)))
        avg_bleu = np.mean(bleu_scores) if bleu_scores else 0
        print(f"Epoch {epoch + 1} - Avg BLEU Score: {avg_bleu:.4f}")

# Split validation data
val_split = 0.15
val_size = int(len(encoder_input) * val_split)
train_idx = len(encoder_input) - val_size
train_data = (encoder_input[:train_idx], decoder_input[:train_idx], decoder_output_exp[:train_idx])
val_data = (encoder_input[train_idx:], decoder_input[train_idx:], decoder_output_exp[train_idx:])

print("Model summary:")
model.summary()

# --- Reverse word index for decoding ---
index_word = {idx: word for word, idx in tokenizer.word_index.items()}
index_word[0] = ''

# --- Training ---
print("Starting training...")
history = model.fit(
    [train_data[0], train_data[1]], train_data[2],
    batch_size=batch_size, epochs=epochs, validation_data=([val_data[0], val_data[1]], val_data[2]),
    callbacks=[early_stop, lr_scheduler, BLEUCallback(val_data, index_word)], verbose=1
)

# --- Inference models setup ---
encoder_model_inf = tf.keras.Model(encoder_inputs, [encoder_outputs, state_h, state_c])

# Decoder inference model
decoder_state_input_h = tf.keras.Input(shape=(lstm_units * 2,))
decoder_state_input_c = tf.keras.Input(shape=(lstm_units * 2,))
encoder_outputs_inf = tf.keras.Input(shape=(max_len_src, lstm_units * 2))
decoder_inputs_inf = tf.keras.Input(shape=(1,))
dec_emb_inf = dec_emb_layer(decoder_inputs_inf)
dec_emb_inf = layers.BatchNormalization()(dec_emb_inf)
decoder_outputs_inf, state_h_inf, state_c_inf = decoder_lstm(
    dec_emb_inf, initial_state=[decoder_state_input_h, decoder_state_input_c]
)
attention_output_inf = attention_layer(decoder_outputs_inf, encoder_outputs_inf)
decoder_concat_inf = layers.Concatenate(axis=-1)([decoder_outputs_inf, attention_output_inf])
decoder_concat_inf = layers.LayerNormalization()(decoder_concat_inf)
decoder_pred_inf = decoder_dense(decoder_concat_inf)
decoder_model_inf = tf.keras.Model(
    [decoder_inputs_inf, decoder_state_input_h, decoder_state_input_c, encoder_outputs_inf],
    [decoder_pred_inf, state_h_inf, state_c_inf]
)

# --- Reverse word index for decoding ---
index_word = {idx: word for word, idx in tokenizer.word_index.items()}
index_word[0] = ''

# --- Beam search decoding ---
def beam_search_decode(input_seq, beam_width=3, max_length=25, repetition_penalty=1.5):
    try:
        enc_outs, s_h, s_c = encoder_model_inf.predict(input_seq, verbose=0)
    except Exception as e:
        print(f"Encoder prediction error: {e}")
        return "I'm here to listen. How are you feeling?"

    start_token_id = tokenizer.word_index.get('<start>', 1)
    end_token_id = tokenizer.word_index.get('<end>', 2)

    # Initialize beam search
    sequences = [[[], 0.0, [s_h[0:1], s_c[0:1]]]]
    for _ in range(max_length):
        all_candidates = []
        for seq, score, states in sequences:
            if seq and seq[-1] == end_token_id:
                all_candidates.append([seq, score, states])
                continue
            # Use start token if sequence is empty, otherwise use last token
            last_token = np.array([[seq[-1] if seq else start_token_id]])
            try:
                predictions, h, c = decoder_model_inf.predict(
                    [last_token, states[0], states[1], enc_outs], verbose=0
                )
                probs = predictions[0, 0, :vocab_size]  # Explicitly limit to vocab_size
                probs[0] = 0  # No padding
                if start_token_id < len(probs):
                    probs[start_token_id] = 0  # No start token in output
                # Filter sequence tokens to valid range
                valid_seq = [t for t in seq[1:] if t < vocab_size and t > 0]
                for token_id in set(valid_seq):
                    probs[token_id] /= repetition_penalty  # Apply penalty only to valid tokens
                top_indices = np.argsort(probs)[-beam_width:]
                for idx in top_indices:
                    if idx < vocab_size:  # Ensure output token is within vocab_size
                        new_seq = seq + [int(idx)]
                        new_score = score + np.log(probs[idx] + 1e-10)
                        all_candidates.append([new_seq, new_score, [h, c]])
            except Exception as e:
                print(f"Decoding error: {e}")
                continue
        if not all_candidates:  # If no valid candidates, break to avoid empty sequence error
            break
        sequences = sorted(all_candidates, key=lambda x: x[1], reverse=True)[:beam_width]

    # Select best sequence
    best_seq = sequences[0][0] if sequences else []
    words = [index_word.get(id, '').strip().lower() for id in best_seq if id not in [0, start_token_id, end_token_id]]
    words = [w for i, w in enumerate(words) if w and len(w) > 1 and (i == 0 or w != words[i-1])]
    if words:
        words[0] = words[0].capitalize()
        response = ' '.join(words).strip()
        if response and response[-1] not in '.!?':
            response += '.'
        response = response.replace(' i ', ' I ').replace(' im ', " I'm ").replace(' ill ', " I'll ").replace(' ive ', " I've ")
    else:
        response = ""
    return response if len(response) > 10 else "I understand you're going through something difficult. Can you tell me more?"
# --- Affirmations and fallbacks ---
affirmations = [
    "You are stronger than you realize, and you have the courage to face any challenge.",
    "Every new day brings fresh possibilities for growth and healing.",
    "Your resilience is inspiring, and you've overcome so much already.",
    "You deserve kindness and support, especially from yourself.",
]
crisis_keywords = [
    'suicide', 'kill myself', 'end my life', 'hopeless', 'worthless',
    "can't go on", 'harm myself', 'hurt myself', 'want to die',
    'no point living', 'better off dead', 'end it all'
]
fallbacks = [
    "I'm here to listen. What's on your mind?",
    "It sounds like you're carrying a lot. Want to share more?",
    "Your feelings matter. Can you tell me more about what's going on?",
]

# --- Emotion detection with pre-trained model ---
from transformers import pipeline
# Modify the sentiment_analyzer initialization
sentiment_analyzer = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english", device=0)  # Use GPU (device=0) if available
# --- Main chatbot response generation function ---
def generate_response(user_input):
    user_lower = clean_text(user_input).lower()

    # Crisis detection
    for keyword in crisis_keywords:
        if keyword in user_lower:
            return ("I'm deeply concerned about what you're sharing. If you're having thoughts of self-harm, "
                    "please reach out to a crisis hotline or a trusted person immediately. "
                    "In the US, you can call 988 for the Suicide & Crisis Lifeline. You are not alone.")

    # Affirmation request
    if any(word in user_lower for word in ['affirmation', 'motivate me', 'encourage me', 'positive']):
        return np.random.choice(affirmations)

    # Emotion detection
    try:
        sentiment = sentiment_analyzer(user_input)[0]
        emotion = sentiment['label'].lower()
        confidence = sentiment['score']
    except:
        emotion, confidence = 'neutral', 0.5

    # Generate model response
    try:
        seq = tokenizer.texts_to_sequences([user_input])
        if not seq or not seq[0]:
            return np.random.choice(fallbacks)
        pad_input = pad_sequences(seq, maxlen=max_len_src, padding='post')
        response_text = beam_search_decode(pad_input)
        if response_text and len(response_text.split()) >= 4:
            return response_text
    except Exception as e:
        print(f"Error generating response: {e}")

    # Emotion-aware fallbacks
    if emotion == 'negative' and confidence > 0.7:
        if any(word in user_lower for word in ['sad', 'depressed', 'down', 'upset', 'crying']):
            return np.random.choice([
                "I'm here for you. It sounds like you're feeling really down.",
                "Sadness can feel heavy. Want to share what's been going on?",
            ])
        elif any(word in user_lower for word in ['anxious', 'worried', 'stressed']):
            return np.random.choice([
                "Anxiety can be tough. What's been making you feel this way?",
                "I hear you're stressed. Let's talk about what's on your mind.",
            ])
    return np.random.choice(fallbacks)

# --- Interactive chatbot loop ---
print("\n" + "="*50)
print("Mental Health Support Chatbot is ready!")
print("Type 'exit', 'quit', or 'stop' to end the conversation.")
print("For affirmations, try saying 'I need some encouragement'")
print("="*50 + "\n")

conversation_count = 0
while True:
    user_input = input("You: ").strip()
    if not user_input:
        continue
    if user_input.lower() in ['exit', 'quit', 'bye', 'goodbye', 'stop']:
        print("Chatbot: Take care! Support is always here when you need it. 💙")
        break
    conversation_count += 1
    response = generate_response(user_input)
    print(f"Chatbot: {response}")
    if conversation_count % 5 == 0:
        print("\n(Reminder: This is an AI chatbot. For professional help, contact a mental health professional.)\n")

# --- Save the model ---
model.save('mental_health_chatbot_improved.keras')
print("\nModel saved to mental_health_chatbot_improved.keras")

# --- Load model and generate response ---
def generate_response_from_saved_model(user_input):
    loaded_model = load_model('mental_health_chatbot_improved.keras',
                              custom_objects={'MultiHeadAttention': layers.MultiHeadAttention})

    # Reconstruct encoder
    encoder_inputs_loaded = loaded_model.input[0]
    enc_emb_layer_loaded = loaded_model.layers[2]
    enc_emb_loaded = enc_emb_layer_loaded(encoder_inputs_loaded)
    enc_bn_loaded = loaded_model.layers[3](enc_emb_loaded)
    encoder_lstm_loaded = loaded_model.layers[4]
    enc_outs, fh, fc, bh, bc = encoder_lstm_loaded(enc_bn_loaded)
    state_h = layers.Concatenate()([fh, bh])
    state_c = layers.Concatenate()([fc, bc])
    encoder_model_inf_loaded = tf.keras.Model(encoder_inputs_loaded, [enc_outs, state_h, state_c])

    # Reconstruct decoder
    decoder_inputs_loaded = loaded_model.input[1]
    dec_emb_layer_loaded = loaded_model.layers[5]
    dec_bn_layer_loaded = loaded_model.layers[6]
    decoder_lstm_loaded = loaded_model.layers[7]
    attention_layer_loaded = loaded_model.layers[8]
    decoder_dense_loaded = loaded_model.layers[10]

    decoder_state_input_h = tf.keras.Input(shape=(lstm_units * 2,))
    decoder_state_input_c = tf.keras.Input(shape=(lstm_units * 2,))
    encoder_outputs_inf = tf.keras.Input(shape=(max_len_src, lstm_units * 2))
    dec_emb_inf = dec_emb_layer_loaded(decoder_inputs_loaded)
    dec_emb_inf = dec_bn_layer_loaded(dec_emb_inf)
    dec_outs, h, c = decoder_lstm_loaded(dec_emb_inf, initial_state=[decoder_state_input_h, decoder_state_input_c])
    att_out = attention_layer_loaded(dec_outs, encoder_outputs_inf)
    dec_concat = layers.Concatenate(axis=-1)([dec_outs, att_out])
    dec_concat = layers.LayerNormalization()(dec_concat)
    dec_pred = decoder_dense_loaded(dec_concat)
    decoder_model_inf_loaded = tf.keras.Model(
        [decoder_inputs_loaded, decoder_state_input_h, decoder_state_input_c, encoder_outputs_inf],
        [dec_pred, h, c]
    )

    seq = tokenizer.texts_to_sequences([user_input])
    pad_input = pad_sequences(seq, maxlen=max_len_src, padding='post')
    response_text = beam_search_decode( pad_input)
    return response_text

# --- Sample usage ---
user_query = "I'm feeling really going to die"
response = generate_response(user_query)
print(f"You: {user_query}")
print(f"Chatbot: {response}")

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

combined_dataset.json: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/3512 [00:00<?, ? examples/s]

README.md:   0%|          | 0.00/274 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cuda:0



Mental Health Support Chatbot is ready!
Type 'exit', 'quit', or 'stop' to end the conversation.
For affirmations, try saying 'I need some encouragement'

You: hi
Chatbot: Values first most first attempt is are is to people is emotional never cry crying never.
You: console me.
Chatbot: If there in family family is people is really feel never is never say your never cry therapy is.
You: affirmation
Chatbot: You are stronger than you realize, and you have the courage to face any challenge.
You: i am sad
Chatbot: Inviting in attempt under family youve first is your first fears sometimes never.
You: bye
Chatbot: Take care! Support is always here when you need it. 💙

Model saved to mental_health_chatbot_improved.keras
You: I'm feeling really going to die
Chatbot: Jobs in they really really feels really small never really never about never about crying is.
